# Training Emotion Classification Model

## Step 1: Setup Environment

In [2]:
# Install required packages
!pip install tensorflow pandas numpy matplotlib seaborn scikit-learn opencv-python albumentations

In [66]:
import os
import glob
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image, ImageFile
import time
from tqdm import tqdm
import albumentations as A  # Powerful augmentation library

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import AUC
from sklearn.metrics import classification_report, roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

warnings.filterwarnings('ignore')


# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Step 3: Load and Prepare Data

Download the [Emotion Recognition Dataset](https://www.kaggle.com/datasets/sujaykapadnis/emotion-recognition-dataset/data) from Kaggle and extract it. 

In [5]:
# Define the path to the CSV file relative to the notebooks folder
csv_path = os.path.abspath(os.path.join(".", "data", "data.csv"))
df = pd.read_csv(csv_path)
df = df[~df['label'].str.contains('Ahegao', case=False, na=False)]
print(df.head())

   Unnamed: 0                                               path     label
0           0  Surprise/1bd930d6a1c717c11be33db74823f661cb53f...  Surprise
1           1       Surprise/cropped_emotions.100096~12fffff.png  Surprise
2           2  Surprise/0df0e470e33093f5b72a8197fa209d684032c...  Surprise
3           3       Surprise/cropped_emotions.260779~12fffff.png  Surprise
4           4       Surprise/cropped_emotions.263616~12fffff.png  Surprise


## Step 3.1: Get rid of bad rows

In [9]:
# Allow truncated images to load (optional)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Define the correct base images directory
base_images_path = os.path.abspath("StudentEngagementAnalyzer/data/images")

# Remove duplicate "StudentEngagementAnalyzer" occurrences if present
cwd = os.getcwd()  # Get current working directory
if "StudentEngagementAnalyzer" in cwd:
    base_images_path = os.path.abspath(os.path.join(cwd, "data/images"))

print(f"✅ Base Image Path Set To: {base_images_path}")

print(f"Initial number of rows: {df.shape[0]}")

# Update chunk size and timer based on request
chunk_size = 400  
valid_rows = [] 

# Iterate through the DataFrame in chunks
for start in range(0, len(df), chunk_size):
    end = start + chunk_size
    chunk = df.iloc[start:end]
    print(f"Processing rows {start} to {end}...")

    for idx, row in chunk.iterrows():
        # Ensure the path is appended correctly (remove any duplicate occurrences)
        img_path = os.path.normpath(os.path.join(base_images_path, row["path"]))
        img_path = os.path.abspath(img_path)  # Standardize to absolute path

        # Check if the file actually exists
        if not os.path.isfile(img_path):
            # print(f"❌ NOT FOUND: {img_path}")
            continue  # Skip this row

        # Try to open the image using OpenCV
        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠️ CORRUPT IMAGE (cv2 check failed): {img_path}")
            continue  # Skip this row

        # If image is valid, store the absolute path
        row["full_path"] = img_path
        valid_rows.append(row)  

    # Sleep between chunks to prevent system overload
    time.sleep(2)  # Reduced from 3 to 2 seconds per request

# Create a new DataFrame with only valid rows
df_cleaned = pd.DataFrame(valid_rows)

# Print final row count
print(f"Total rows after cleaning: {df_cleaned.shape[0]}")

# Save cleaned dataset
cleaned_csv_path = os.path.abspath("data/cleaned_data.csv")
df_cleaned.to_csv(cleaned_csv_path, index=False)
print(f"✅ Cleaned dataset saved as {cleaned_csv_path}")


✅ Base Image Path Set To: /Users/parhamhajzavar/Documents/Seattle/UW/Classes/Winter 2024/522/Project/StudentEngagementAnalyzer/data/images
Initial number of rows: 14248
Processing rows 0 to 400...
Processing rows 400 to 800...
Processing rows 800 to 1200...
Processing rows 1200 to 1600...
Processing rows 1600 to 2000...
Processing rows 2000 to 2400...


Premature end of JPEG file


Processing rows 2400 to 2800...
Processing rows 2800 to 3200...
Processing rows 3200 to 3600...
Processing rows 3600 to 4000...
Processing rows 4000 to 4400...
Processing rows 4400 to 4800...
Processing rows 4800 to 5200...
Processing rows 5200 to 5600...
Processing rows 5600 to 6000...
Processing rows 6000 to 6400...
Processing rows 6400 to 6800...
Processing rows 6800 to 7200...
Processing rows 7200 to 7600...
Processing rows 7600 to 8000...
Processing rows 8000 to 8400...
Processing rows 8400 to 8800...
Processing rows 8800 to 9200...
Processing rows 9200 to 9600...
Processing rows 9600 to 10000...
Processing rows 10000 to 10400...
Processing rows 10400 to 10800...
Processing rows 10800 to 11200...
Processing rows 11200 to 11600...
Processing rows 11600 to 12000...
Processing rows 12000 to 12400...
Processing rows 12400 to 12800...
Processing rows 12800 to 13200...
Processing rows 13200 to 13600...
Processing rows 13600 to 14000...
Processing rows 14000 to 14400...
Total rows after 

## Step 3.2: Augment the dataset

In [13]:
# Define paths
base_images_path = os.path.abspath("data/images")
augmented_images_path = os.path.abspath("data/augmented_images")

# Ensure base augmented directory exists
os.makedirs(augmented_images_path, exist_ok=True)

# Load the cleaned dataset
df_cleaned_path = os.path.abspath("data/cleaned_data.csv")
df = pd.read_csv(df_cleaned_path)

# Augmentation pipeline
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.2)
])

# Store new data paths
augmented_rows = []

print(f"Starting augmentation for {len(df)} images...")

for idx, row in df.iterrows():
    img_path = row["full_path"]
    emotion_folder = row["label"]  # Extract the emotion label

    # Read image
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Skipping {img_path} (Could not load image)")
        continue  # Skip invalid images
    
    # Convert from BGR (OpenCV) to RGB (for augmentation)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Ensure the emotion subdirectory exists in `augmented_images/`
    emotion_augmented_path = os.path.join(augmented_images_path, emotion_folder)
    os.makedirs(emotion_augmented_path, exist_ok=True)

    # Generate multiple augmented versions
    for i in range(3):  # Create 3 augmented versions per image
        augmented = augmentations(image=img)["image"]

        # Convert back to BGR for saving
        augmented = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

        # Generate new filename with "augmented-" prefix
        orig_filename = os.path.basename(img_path)
        new_filename = f"augmented-{i}-{orig_filename}"
        new_img_path = os.path.join(emotion_augmented_path, new_filename)  # Save under correct emotion folder

        # Save augmented image
        cv2.imwrite(new_img_path, augmented)

        # Store new entry for CSV (using relative path)
        augmented_rows.append({"path": f"augmented_images/{emotion_folder}/{new_filename}", "label": row["label"]})

    # Progress update
    if idx % 100 == 0:
        print(f"Processed {idx}/{len(df)} images...")

# Create augmented DataFrame
df_augmented = pd.DataFrame(augmented_rows)

# Combine original and augmented data
df_final = pd.concat([df, df_augmented], ignore_index=True)

# Save the new dataset
augmented_csv_path = os.path.abspath("data/augmented_data.csv")
df_final.to_csv(augmented_csv_path, index=False)

print(f"✅ Augmentation complete! Augmented dataset saved as {augmented_csv_path}")
print(f"📂 Augmented images are stored inside: {augmented_images_path}/[Emotion]/")


Starting augmentation for 10977 images...
Processed 0/10977 images...
Processed 100/10977 images...
Processed 200/10977 images...
Processed 300/10977 images...
Processed 400/10977 images...
Processed 500/10977 images...
Processed 600/10977 images...
Processed 700/10977 images...
Processed 800/10977 images...
Processed 900/10977 images...
Processed 1000/10977 images...
Processed 1100/10977 images...
Processed 1200/10977 images...
Processed 1300/10977 images...
Processed 1400/10977 images...
Processed 1500/10977 images...
Processed 1600/10977 images...
Processed 1700/10977 images...
Processed 1800/10977 images...
Processed 1900/10977 images...


Premature end of JPEG file


Processed 2000/10977 images...
Processed 2100/10977 images...
Processed 2200/10977 images...
Processed 2300/10977 images...
Processed 2400/10977 images...
Processed 2500/10977 images...
Processed 2600/10977 images...
Processed 2700/10977 images...
Processed 2800/10977 images...
Processed 2900/10977 images...
Processed 3000/10977 images...
Processed 3100/10977 images...
Processed 3200/10977 images...
Processed 3300/10977 images...
Processed 3400/10977 images...
Processed 3500/10977 images...
Processed 3600/10977 images...
Processed 3700/10977 images...
Processed 3800/10977 images...
Processed 3900/10977 images...
Processed 4000/10977 images...
Processed 4100/10977 images...
Processed 4200/10977 images...
Processed 4300/10977 images...
Processed 4400/10977 images...
Processed 4500/10977 images...
Processed 4600/10977 images...
Processed 4700/10977 images...
Processed 4800/10977 images...
Processed 4900/10977 images...
Processed 5000/10977 images...
Processed 5100/10977 images...
Processe

## Step 4: Create and Compile Model

Use MobileNetV2 as base model and add custom layers for emotion classification

In [23]:
# Define image size and batch size
IMG_SIZE = (200, 200)
BATCH_SIZE = 32
NUM_CLASSES = 5  # Adjust based on your dataset

# Define emotion class names
emotion_classes = ["Angry", "Happy", "Neutral", "Sad", "Surprise"]

# Define dataset paths
train_data_path = "data/augmented_images"

# Image data generator (includes real-time augmentation)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2  # 20% validation split
)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Define F1 Score metric
def f1_score(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(y_true * y_pred)
    precision = tp / (tf.reduce_sum(y_pred) + tf.keras.backend.epsilon())
    recall = tp / (tf.reduce_sum(y_true) + tf.keras.backend.epsilon())
    return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

# Define the model
def create_model(input_shape=(200, 200, 3)):
    # Create and compile the MobileNetV2 model
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)  # Adjust to num classes

    model = Model(inputs=base_model.input, outputs=outputs)
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', AUC(name='auc'), f1_score]
    )

    return model

# Create and summarize the model
model = create_model()
model.summary()

# Define training parameters
EPOCHS = 15  # Adjust as needed

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    verbose=1
)

# Unfreeze some layers for fine-tuning
for layer in model.layers[:100]:  
    layer.trainable = False  # Freeze first 100 layers
for layer in model.layers[100:]:  
    layer.trainable = True  # Fine-tune deeper layers

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(name='auc'), f1_score]
)

# Fine-tune model for a few more epochs
FINE_TUNE_EPOCHS = 5
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=FINE_TUNE_EPOCHS,
    verbose=1
)

# Evaluate the model
print("Evaluating model on validation set...")
val_loss, val_acc, val_auc, val_f1 = model.evaluate(val_generator)

print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")

# Generate predictions
y_pred_prob = model.predict(val_generator)  # Get probability predictions
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert to class labels
y_true = val_generator.classes  # True labels

# Classification Report
print("🔹 Classification Report:")
print(classification_report(y_true, y_pred, target_names=emotion_classes))

# Compute AUC Score (macro-average)
auc_score = roc_auc_score(tf.keras.utils.to_categorical(y_true, num_classes=NUM_CLASSES), y_pred_prob, multi_class="ovr")
print(f"🔹 ROC-AUC Score: {auc_score:.4f}")


Found 26349 images belonging to 5 classes.
Found 6584 images belonging to 5 classes.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 100, 100,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 100, 100,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 100, 100,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 100, 100,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 100, 100,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 100, 100,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 100, 100,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 100, 100,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 100, 100,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 100, 100,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 100, 100,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 101, 101,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 50, 50,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 50, 50,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 50, 50,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 50, 50,    │      2,304 │ block_1_depthwis

 Total params: 2,588,229 (9.87 MB)

 Trainable params: 329,733 (1.26 MB)

...

824/824 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.5601 - auc: 0.8516 - f1_score: 0.4959 - loss: 1.0713

2025-02-03 00:50:08.079471: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


824/824 ━━━━━━━━━━━━━━━━━━━━ 169s 194ms/step - accuracy: 0.5602 - auc: 0.8516 - f1_score: 0.4960 - loss: 1.0711 - val_accuracy: 0.7132 - val_auc: 0.9388 - val_f1_score: 0.7039 - val_loss: 0.6848
Epoch 2/5
824/824 ━━━━━━━━━━━━━━━━━━━━ 153s 186ms/step - accuracy: 0.7509 - auc: 0.9498 - f1_score: 0.7380 - loss: 0.6266 - val_accuracy: 0.7861 - val_auc: 0.9619 - val_f1_score: 0.7785 - val_loss: 0.5465
Epoch 3/5
824/824 ━━━━━━━━━━━━━━━━━━━━ 154s 187ms/step - accuracy: 0.8278 - auc: 0.9735 - f1_score: 0.8220 - loss: 0.4542 - val_accuracy: 0.7269 - val_auc: 0.9430 - val_f1_score: 0.7222 - val_loss: 0.6669
Epoch 4/5
824/824 ━━━━━━━━━━━━━━━━━━━━ 156s 189ms/step - accuracy: 0.8791 - auc: 0.9859 - f1_score: 0.8755 - loss: 0.3295 - val_accuracy: 0.7705 - val_auc: 0.9568 - val_f1_score: 0.7648 - val_loss: 0.5791
Epoch 5/5
824/824 ━━━━━━━━━━━━━━━━━━━━ 154s 186ms/step - accuracy: 0.9112 - auc: 0.9918 - f1_score: 0.9098 - loss: 0.2470 - val_accuracy: 0.8003 - val_auc: 0.9644 - val_f1_score: 0.7988 - va

2025-02-03 01:01:23.331244: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


206/206 ━━━━━━━━━━━━━━━━━━━━ 26s 119ms/step
🔹 Classification Report:
              precision    recall  f1-score   support

       Angry       0.13      0.09      0.11       787
       Happy       0.19      0.20      0.19      1157
     Neutral       0.30      0.35      0.32      1961
         Sad       0.27      0.22      0.25      1939
    Surprise       0.11      0.14      0.13       740

    accuracy                           0.23      6584
   macro avg       0.20      0.20      0.20      6584
weighted avg       0.23      0.23      0.23      6584

🔹 ROC-AUC Score: 0.4974


## Step 5: Prepare Data Generator

Create a data generator to efficiently load and preprocess images:
- Convert labels from Strings to Integers before one-hot encoding

In [62]:
# num_batches = len(list(dataset))
# print(f"📌 Total Batches in Dataset: {num_batches}")


In [58]:



# Image size
INPUT_SHAPE = (200, 200)

def preprocess_dataframe(df):
    """Ensure paths are correct and labels are encoded."""
    if 'full_path' not in df.columns:
        raise ValueError("❌ Missing 'full_path' column in df!")
    
    # Convert emotion labels to numeric values
    label_encoder = LabelEncoder()
    df['label_encoded'] = label_encoder.fit_transform(df['label'])  # Convert text labels to numbers
    
    print(f"✅ Label Mapping: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")
    
    return df, label_encoder

def load_and_preprocess_image(image_path, label):
    """Load and preprocess a single image."""
    try:
        img = tf.io.read_file(image_path)  # Read file
        img = tf.image.decode_jpeg(img, channels=3)  # Decode image
        img = tf.image.resize(img, INPUT_SHAPE)  # Resize
        img = tf.keras.applications.mobilenet_v2.preprocess_input(img)  # Normalize
        return img, label
    except Exception as e:
        print(f"⚠️ Error loading {image_path}: {e}")
        return None, None  # Skip bad images

def preprocess_data(df):
    """Preprocess images and create a dataset with correct labels."""
    
    # Ensure paths and labels are correctly stored
    image_paths = df["full_path"].values
    labels = df["label_encoded"].values

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Load and preprocess images
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Ensure valid images
    dataset = dataset.filter(lambda x, y: tf.reduce_any(tf.math.is_finite(x)))

    dataset = dataset.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=len(label_encoder.classes_))))

    dataset = dataset.shuffle(buffer_size=1000).batch(32, drop_remainder=True).prefetch(tf.data.AUTOTUNE)


    return dataset

# Preprocess DataFrame and Verify Labels
df, label_encoder = preprocess_dataframe(df)

# Preprocess Dataset
dataset = preprocess_data(df)

# Split Dataset
train_size = int(0.8 * len(df))
train_dataset = dataset.take(train_size).batch(32, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val_dataset = dataset.skip(train_size).batch(32, drop_remainder=True).prefetch(tf.data.AUTOTUNE)


print(f"✅ Dataset split: {train_size} training samples, {len(df) - train_size} validation samples.")

for img, label in dataset.take(10):
    print(f"Image shape: {img.shape}, Label: {label.numpy()}")


✅ Label Mapping: {'Angry': 0, 'Happy': 1, 'Neutral': 2, 'Sad': 3, 'Surprise': 4}
✅ Dataset split: 8781 training samples, 2196 validation samples.
Image shape: (32, 200, 200, 3), Label: [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
Image shape: (32, 200, 200, 3), Label: [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 

2025-02-03 02:08:04.954029: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [68]:
def preprocess_data(df):
    """Preprocess images and create a dataset with correct labels."""
    
    # Ensure paths and labels are correctly stored
    image_paths = df["full_path"].values
    labels = df["label_encoded"].values

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Load and preprocess images
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Ensure valid images
    dataset = dataset.filter(lambda x, y: tf.reduce_any(tf.math.is_finite(x)))

    # ✅ Fix: Convert labels to one-hot encoding **AFTER filtering**
    dataset = dataset.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=len(label_encoder.classes_))))

    # ✅ Fix: Shuffle and batch correctly
    dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

    return dataset


In [70]:
# Check label distribution
import pandas as pd

print("Label counts:")
print(df["label"].value_counts())  # Check if some classes are missing

print("Encoded label mapping:")
print(dict(enumerate(label_encoder.classes_)))  # Ensure encoding matches



# preprocess_dataframe(df)
# for img, label in dataset.take(3):
#     print(img.shape, label.numpy())


Label counts:
label
Neutral     3269
Sad         3233
Happy       1928
Angry       1313
Surprise    1234
Name: count, dtype: int64
Encoded label mapping:
{0: 'Angry', 1: 'Happy', 2: 'Neutral', 3: 'Sad', 4: 'Surprise'}


## Step 6: Train the Model

In [76]:
# Setup callbacks for better training
callbacks = [
    # Save the best model
    ModelCheckpoint(
        'best_model.h5',  # Save model to this file
        monitor='val_accuracy',  # Watch validation accuracy
        save_best_only=True,  # Only save if better than previous
        mode='max',  # Higher accuracy is better
        verbose=1
    ),
    # Stop if not improving
    EarlyStopping(
        monitor='val_loss',  # Watch validation loss
        patience=10,  # Wait 10 epochs for improvement
        restore_best_weights=True,  # Use best weights when done
        verbose=1
    ),
    # Reduce learning rate when stuck
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,  # Reduce LR by 90%
        patience=5,  # Wait 5 epochs before reducing
        min_lr=1e-6,  # Don't go below this LR
        verbose=1
    )
]

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,  # Maximum number of epochs
    batch_size=32,  # Process 32 images at a time
    callbacks=callbacks,
    verbose=1  # Show progress
)

Epoch 1/50


ValueError: as_list() is not defined on an unknown TensorShape.

## Step 7: Visualize Training Results

In [ ]:
def plot_training_results(history):
    """Plot training metrics"""
    # Set style
    sns.set_style("whitegrid")
    sns.set_palette("husl")
    
    # Create figure
    fig = plt.figure(figsize=(15, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot results
plot_training_results(history)

## Step 8: Save the Model

The model has already been saved as 'best_model.h5' through the ModelCheckpoint callback during training. You can now copy this file to your Student Engagement Analysis project directory.

To verify the model was saved:

In [ ]:
import os
if os.path.exists('best_model.h5'):
    print("Model saved successfully!")
    print(f"Model file size: {os.path.getsize('best_model.h5') / (1024*1024):.2f} MB")
else:
    print("Error: Model file not found!")